In [2]:
# Run in terminal or command prompt
# python3 -m spacy download en



import numpy as np
import pandas as pd
import sklearn
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline



/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [24]:
#collection data is being loaded from
collection =  "lda_info"
#documents data is being loaded from
documents = [
  "_id",
  "dc:description:eng",
  "dc:title:eng"
]#read file
f = open("avois_words.txt", "r")
#this is a list of words that are not relevant to the topic
stopwords = f.read().splitlines()
f.close()

In [28]:
import os
from pymongo import MongoClient

#Function to convert dictionary into a sorted list with max_qty 
pipeline = [
    {"$project": {"dc:description:eng":1, "dc:title:eng":1}},

]

def get_mongo_conn(host='localhost', port=27018):
  """ 
  retorna la conexion a la base de datos mongo en caso
  de ser posible
  """
  username_db = "mongo-root"
  password_db = "horus.mongo.2020"
  try:
    return MongoClient(host, port, username=username_db, password=password_db)
  except Exception as e:
    print("No se pudo establecer conexion con la base de datos mongo")
    print(e)
    return None
  
client = get_mongo_conn()
db = client['VT']

def get_data():
  data = []
  cnt = 0
  for document in db[collection].aggregate(pipeline):
    new_doc = []
    for(key, value) in document.items():
      if(key in documents):
        new_doc.append(key + ":" + str(value))
    data.append(new_doc)
  return data
data = get_data()

In [29]:
#TOKENIZE WORDS
def tokenize_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(tokenize_words(data))

print(data_words[:1])

[['dc', 'title', 'eng', 'systematic', 'evaluation', 'of', 'jatropha', 'curcas', 'oil', 'obtention', 'by', 'mechanical', 'and', 'solvent', 'extraction', 'dc', 'description', 'eng', 'current', 'biodiesel', 'production', 'relies', 'mostly', 'on', 'edible', 'oils', 'soybean', 'palm', 'rapeseed', 'which', 'is', 'major', 'drawback', 'for', 'the', 'process', 'as', 'raw', 'materials', 'represent', 'the', 'major', 'part', 'of', 'the', 'final', 'costs', 'of', 'the', 'biofuel', 'this', 'also', 'brings', 'out', 'concerns', 'about', 'food', 'and', 'feed', 'security', 'in', 'the', 'last', 'years', 'jatropha', 'curca', 'oil', 'has', 'obtained', 'attention', 'as', 'an', 'alternative', 'oleochemical', 'feedstock', 'because', 'it', 'is', 'perennial', 'crop', 'able', 'to', 'produce', 'up', 'to', 'gallon', 'per', 'hectare', 'year', 'of', 'non', 'edible', 'oil', 'even', 'though', 'several', 'authors', 'have', 'reported', 'process', 'conditions', 'and', 'yields', 'on', 'variety', 'of', 'extraction', 'method

In [30]:
#LEMMATIZATION
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['evaluation oil obtention mechanical solvent extraction dc description current biodiesel production rely mostly edible oil soybean rapeseed major drawback process raw material represent major part final cost biofuel also bring concern food feed security last year curca oil obtain attention alternative oleochemical feedstock perennial crop able produce gallon hectare year edible oil even several author report process condition yield variety extraction method jatropha oil few present detail overall process industrial standpoint also most assess impact extraction process oil quality technical requirement biodiesel production regard work describe development jatropha oil production process involve mechanical solvent extraction effect mechanical extraction parameter such speed screw diameter temperature discharge one analyze oil yield saponification index quality parameter expeller power consumption extraction efficiency study solvent extraction experiment evaluate effect nature solvent us

In [31]:
# Create document word matrix
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=stopwords,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)
print(data_vectorized.shape)

(742, 982)


/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['activated', 'cytokine', 'diet', 'induced', 'line', 'lisis', 'perzonalized', 'realiz', 'spin', 'strategies'] not in stop_words.
  warnings.warn(


In [32]:
#SPARSE DATA
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  4.0525963296205 %


In [33]:
# Manually build and fit LDA model  ** JUST TO SEE HOW IT WORKS ** we will use GridSearchCV to find the best model
#lda_model = LatentDirichletAllocation(n_components=3,               # Number of topics
#                                      max_iter=10,               # Max learning iterations
#                                      learning_method='online',   
#                                      random_state=100,          # Random state
#                                      batch_size=128,            # n docs in each learning iter
#                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
#                                      n_jobs = -1,               # Use all available CPUs
#                                     )
#lda_output = lda_model.fit_transform(data_vectorized)
#
#print(lda_model)  # Model attributes
#MODEL PERFORMANCE AND LOG_LIKELIHOOD
# Log Likelyhood: Higher the better
#print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
#print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
# higher log-likelihood and lower perplexity
#pprint(lda_model.get_params())

In [54]:
#GRID SEARCH TO FIND BEST n_components
#TARDA 11 min maomenos
# Define Search Param
search_params = {'n_components': [4], 
                 'learning_decay': [0.5],
                 'max_iter':[100],            # Max learning iterations
                 'evaluate_every': [20],
                 'batch_size':[128],          # n docs in each learning iter
                 'n_jobs':[-1],               # Use all available CPUs
                 }

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(estimator=lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'batch_size': [128], 'evaluate_every': [20],
                         'learning_decay': [0.5], 'max_iter': [100],
                         'n_components': [4], 'n_jobs': [-1]})

In [55]:
#GET BEST MODEL
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'batch_size': 128, 'evaluate_every': 20, 'learning_decay': 0.5, 'max_iter': 100, 'n_components': 4, 'n_jobs': -1}
Best Log Likelihood Score:  -61432.13843389174
Model Perplexity:  546.9265849021027


In [56]:
pyLDAvis.enable_notebook()
#panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds="mmds")
panel

/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.161280  0.068125       1        1  29.305910
0      0.102811  0.196884       2        1  29.089771
2      0.165684 -0.063234       3        1  22.364324
1     -0.107216 -0.201775       4        1  19.239995, topic_info=            Term        Freq       Total Category  logprob  loglift
440        image  343.000000  343.000000  Default  30.0000  30.0000
596      network  399.000000  399.000000  Default  29.0000  29.0000
136      cluster  200.000000  200.000000  Default  28.0000  28.0000
34     algorithm  294.000000  294.000000  Default  27.0000  27.0000
288     emission  162.000000  162.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
237       design   71.263858  269.772822   Topic4  -4.7729   0.3170
220        datum   72.008229  339.332121   Topic4  -4.7625   0.0980
771       result   70.318057  496.877966   Topic4  -4.7862  -0.3071
949     variable   52.718975   99.956716   Topic4  -5.0743   1.0084
236  description   57.245520  477.175618   Topic4  -4.9919  -0.4724

[263 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        4  0.976561      acid
30        3  0.851379     agent
30        4  0.135447     agent
32        1  0.988379       air
33        4  0.979961   alcohol
...     ...       ...       ...
955       1  0.988881   vehicle
961       3  0.972329   virtual
963       2  0.994200    visual
967       4  0.996220     water
974       3  0.962070  wireless

[381 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

In [57]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    print(lda_model.components_.shape)
    print(lda_model.components_)
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=20)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords
df_n_relevant_topic_keywords = df_topic_keywords

(4, 982)
[[ 8.52775855  4.88021937  3.99403806 ...  0.25757609  4.85945873
   0.25005046]
 [ 2.77179111 17.38804377  0.25007201 ... 12.68320144 24.80610075
   8.7742931 ]
 [12.99706665  5.10367954  0.25609241 ...  5.95479547  5.07188108
   0.25187691]
 [ 6.70338368 13.62805731 34.49979752 ... 37.10442699  0.26255944
   8.72377954]]


/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [58]:
#dominant topic in each document -> Assing a topic to each document
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
print("len topicnames",len(topicnames))
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
print("len docsnames",len(docnames))
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics



len topicnames 4
len docsnames 742


,Topic0,Topic1,Topic2,Topic3,dominant_topic
Doc0,0.000000,0.990000,0.000000,0.000000,1
Doc1,0.010000,0.010000,0.980000,0.010000,2
Doc2,0.010000,0.010000,0.010000,0.980000,3
Doc3,0.010000,0.010000,0.880000,0.100000,2
Doc4,0.010000,0.010000,0.980000,0.010000,2
Doc5,0.010000,0.010000,0.120000,0.870000,3
Doc6,0.010000,0.010000,0.010000,0.970000,3
Doc7,0.010000,0.010000,0.980000,0.010000,2
Doc8,0.010000,0.010000,0.310000,0.680000,3
Doc9,0.010000,0.010000,0.980000,0.010000,2


In [59]:
#TOPICS KEYWORDS (PESO DE CADA PALABRA EN CADA TOPIC)
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

/home/nivalderramas/dev/horus/lda/venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ability,abstract,academic,access,accomplish,accord,account,accuracy,accurate,achieve,...,wide,widely,wireless,word,workshop,world,write,year,yield,zone
Topic0,8.527759,4.880219,3.994038,5.110671,7.002187,30.430120,6.783840,35.970601,0.285995,25.663870,...,0.275104,1.945263,0.257411,44.664639,0.266538,0.259824,0.257680,0.257576,4.859459,0.250050
Topic1,2.771791,17.388044,0.250072,0.250424,1.703663,11.704027,3.255634,8.085018,0.251788,15.159764,...,6.441608,6.289473,0.250105,0.250301,0.252257,10.819451,0.250016,12.683201,24.806101,8.774293
Topic2,12.997067,5.103680,0.256092,19.671774,4.041785,5.938177,20.953187,11.348755,3.646248,17.785610,...,7.157460,13.288248,15.241311,0.253980,0.251464,18.005934,2.227521,5.954795,5.071881,0.251877
Topic3,6.703384,13.628057,34.499798,15.967131,0.252366,27.927676,21.007339,4.595626,11.815969,31.390756,...,8.125828,5.477015,0.251173,2.831080,14.229741,24.914791,12.264782,37.104427,0.262559,8.723780


In [60]:
# PREDICT TOPIC FOR NEW TEXT
# Define function to predict topic for a given text document.

def predict_topic(text, nlp=nlp):

    # Step 1: Clean with simple_preprocess
    tokenized_words = list(tokenize_words(text))

    # Step 2: Lemmatize
    lemmatized_words = lemmatization(tokenized_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    vectorized_words = vectorizer.transform(lemmatized_words)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(vectorized_words)
    topic = df_n_relevant_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["algorithm to predict cancer on colombian computers"]
topic, prob_scores = predict_topic(text = mytext)
print("prob_scores",prob_scores)
print(topic)

prob_scores [[0.63078843 0.05036647 0.0654006  0.2534445 ]]
['image', 'algorithm', 'base', 'cluster', 'result', 'datum', 'description', 'paper', 'problem', 'feature', 'representation', 'learning', 'learn', 'neural', 'strategy', 'visual', 'classification', 'information', 'genetic', 'evolutionary']


In [61]:
#Get similar documents for a given text
from sklearn.metrics.pairwise import euclidean_distances


def similar_documents(text, doc_topic_probs, documents = data, nlp=nlp, top_n=5, verbose=False):
    topic, x  = predict_topic(text)
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    doc_ids = np.argsort(dists)[:top_n]
    if verbose:        
        print("Topic KeyWords: ", topic)
        print("Topic Prob Scores of text: ", np.round(x, 1))
        print("Most Similar Doc's Probs:  ", np.round(doc_topic_probs[doc_ids], 1))
    return doc_ids, np.take(documents, doc_ids)

In [62]:
# Get similar documents
def get_n_similar_publications(text, N = 1):
  doc_ids, docs = similar_documents(text=text, doc_topic_probs=lda_output, documents = data, top_n=N, verbose=True)
  for i in range(len(docs)):
    print("*"*10)
    print(i,docs[i])

In [64]:
get_n_similar_publications(["detection of malicious user in mobile"], N = 15)

Topic KeyWords:  ['network', 'base', 'paper', 'design', 'description', 'environment', 'service', 'software', 'agent', 'result', 'information', 'allow', 'architecture', 'tool', 'programming', 'implementation', 'communication', 'problem', 'control', 'user']
Topic Prob Scores of text:  [[0.1 0.1 0.7 0.1]]
Most Similar Doc's Probs:   [[0.1 0.1 0.7 0.1]
 [0.1 0.  0.8 0.1]
 [0.  0.1 0.8 0.1]
 [0.1 0.1 0.8 0. ]
 [0.1 0.2 0.8 0. ]
 [0.1 0.  0.8 0.1]
 [0.1 0.  0.8 0.2]
 [0.  0.2 0.7 0.1]
 [0.  0.  0.8 0.1]
 [0.  0.  0.8 0.1]
 [0.1 0.  0.9 0. ]
 [0.2 0.  0.8 0. ]
 [0.2 0.  0.8 0. ]
 [0.2 0.  0.8 0. ]
 [0.  0.  0.8 0.2]]
**********
0 dc:title:eng:Certification and International Mobility of Engineering Professionals in Colombia: A Literature Review 
**********
1 dc:description:eng:© 2018 IEEE.South American countries are not in the top of E-Government development, As a proof of this, in the EGovernment Develop Index survey given by the United Nations, no country of the region have been in the top 